In [2]:
import argparse
import torch

import train
import generate

In [ ]:
# train
device = 'cuda'

args = argparse.Namespace()
args.path = '/home/ec2-user/SageMaker/images_brazil_lmdb'
args.iter = 800000
args.batch = 4
args.n_sample = 25
args.size = 256
args.r1 = 10
args.path_regularize = 2
args.path_batch_shrink = 2
args.d_reg_every = 16
args.g_reg_every = 4
args.mixing = 0.9
args.ckpt = 'checkpoint/576500.pt'
args.lr = 0.002
args.channel_multiplier = 2
args.wandb = False
args.local_rank = 0

args.save_ckpt_every = 100
args.save_sample_every = 10
args.name = 'v1'

train.setup_and_run(device, args)

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


load model: checkpoint/576500.pt


d: 1.3528; g: 0.6164; r1: 0.0003; path: 0.0036; mean path: 0.0043:  72%|███████▏  | 576532/800000 [07:43<2543:24:08, 40.97s/it] 

In [3]:
# generate
device = 'cuda'

args = argparse.Namespace()
args.size = 256
args.sample = 1
args.pics = 3
args.truncation = 1
args.truncation_mean = 4096
args.ckpt = 'checkpoint/576500.pt'
args.channel_multiplier = 2

args.latent = 512
args.n_mlp = 8

g_ema = generate.Generator(
    args.size, args.latent, args.n_mlp, channel_multiplier=args.channel_multiplier
).to(device)
checkpoint = torch.load(args.ckpt)

g_ema.load_state_dict(checkpoint['g_ema'], strict=False)

if args.truncation < 1:
    with torch.no_grad():
        mean_latent = g_ema.mean_latent(args.truncation_mean)
else:
    mean_latent = None

generate.generate(args, g_ema, device, mean_latent)


100%|██████████| 3/3 [00:00<00:00,  7.37it/s]
